# Lab 3: Extending Azure SRE Agent with AG-UI

**Duration:** 15 minutes  
**Objective:** Build custom SRE tools that return rich UI component data, serve them via AG-UI protocol, and see beautiful dashboards rendered automatically

---

## What You'll Learn

1. **Create SRE tools** that return structured data optimized for modern UI frameworks
2. **Build a real agent** using Microsoft Agent Framework with these tools
3. **Serve via AG-UI** - expose your agent through the AG-UI protocol
4. **Rich rendering** - see how tool results become charts, cards, tables, and dashboards

---

## Prerequisites

- Completed Lab 1 (Agent Framework basics)
- `.env` file configured with `PROJECT_ENDPOINT`

---

## Architecture Overview

```
┌─────────────────────────────────────────────────────────────────────┐
│                       Your SRE Agent                                │
├─────────────────────────────────────────────────────────────────────┤
│  Tools return UI-ready data with component hints:                   │
│                                                                     │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐    │
│  │  query_metrics  │  │ execute_runbook │  │   get_alerts    │    │
│  │  ┌───────────┐  │  │  ┌───────────┐  │  │  ┌───────────┐  │    │
│  │  │ LineChart │  │  │  │ Stepper   │  │  │  │ DataTable │  │    │
│  │  │ StatCards │  │  │  │ Progress  │  │  │  │ AlertCard │  │    │
│  │  │ Sparkline │  │  │  │ Badge     │  │  │  │ Badge     │  │    │
│  │  └───────────┘  │  │  └───────────┘  │  │  └───────────┘  │    │
│  └─────────────────┘  └─────────────────┘  └─────────────────┘    │
│                                │                                   │
│                    ┌───────────▼───────────┐                       │
│                    │   AG-UI Protocol      │                       │
│                    └───────────┬───────────┘                       │
└────────────────────────────────┼───────────────────────────────────┘
                                 │
              ┌──────────────────┼──────────────────┐
              ▼                  ▼                  ▼
     ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
     │  CopilotKit │    │   Shadcn    │    │  Custom     │
     │   + React   │    │   + Recharts│    │  Dashboard  │
     └─────────────┘    └─────────────┘    └─────────────┘
```

---

## Step 1: Install Dependencies

In [ ]:
%pip install agent-framework agent-framework-azure-ai agent-framework-ag-ui --pre --quiet
%pip install fastapi uvicorn nest-asyncio --quiet

---

## Step 2: Define SRE Tools with Rich UI Component Output

These tools return data structures with **UI component hints** that AG-UI clients can use to render beautiful dashboards automatically.

In [ ]:
import os
import json
from typing import Annotated, Any, Dict, List
from datetime import datetime, timedelta
from pydantic import Field
from dotenv import load_dotenv

from agent_framework import ChatAgent, tool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

load_dotenv()

print("✓ Imports loaded")

In [ ]:
# Tool 1: Query service metrics - returns data optimized for dashboards
@tool
def query_service_metrics(
    service_name: Annotated[str, Field(description="Name of the service to query (e.g., 'payment-service', 'inventory-service')")],
    time_range: Annotated[str, Field(description="Time range for metrics (e.g., '1h', '6h', '24h')")] = "1h"
) -> Dict[str, Any]:
    """
    Query metrics for a service. Returns structured data with UI component hints
    for rendering charts, gauges, and stat cards.
    """
    services = {
        "payment-service": {
            "status": "degraded",
            "status_color": "#f59e0b",  # amber
            "health_score": 62,
            "metrics": [
                {"name": "Request Rate", "key": "request_rate", "value": 1250, "unit": "req/min", "trend": "up", "change": 15, "status": "normal", "icon": "activity"},
                {"name": "Error Rate", "key": "error_rate", "value": 2.5, "unit": "%", "trend": "up", "change": 1.8, "status": "critical", "icon": "alert-triangle", "threshold": 1.0},
                {"name": "P50 Latency", "key": "p50_latency", "value": 45, "unit": "ms", "trend": "stable", "change": 0, "status": "normal", "icon": "clock"},
                {"name": "P99 Latency", "key": "p99_latency", "value": 892, "unit": "ms", "trend": "up", "change": 340, "status": "critical", "icon": "clock", "threshold": 500},
                {"name": "CPU Usage", "key": "cpu_usage", "value": 78, "unit": "%", "trend": "up", "change": 23, "status": "warning", "icon": "cpu", "threshold": 80},
                {"name": "Memory", "key": "memory_usage", "value": 4.2, "unit": "GB", "trend": "up", "change": 0.8, "status": "warning", "icon": "hard-drive", "max": 6}
            ],
            "charts": {
                "latency": {
                    "type": "area",
                    "title": "P99 Latency Over Time",
                    "color": "#ef4444",
                    "gradient": True,
                    "threshold_line": 500,
                    "y_axis": {"label": "Latency (ms)", "min": 0, "max": 1000},
                    "data": [
                        {"time": "14:00", "value": 250, "baseline": 200},
                        {"time": "14:15", "value": 280, "baseline": 200},
                        {"time": "14:30", "value": 450, "baseline": 200},
                        {"time": "14:45", "value": 720, "baseline": 200},
                        {"time": "15:00", "value": 892, "baseline": 200}
                    ]
                },
                "error_rate": {
                    "type": "bar",
                    "title": "Error Rate",
                    "color": "#f59e0b",
                    "threshold_line": 1.0,
                    "y_axis": {"label": "Error %", "min": 0, "max": 5},
                    "data": [
                        {"time": "14:00", "value": 0.5},
                        {"time": "14:15", "value": 0.7},
                        {"time": "14:30", "value": 1.2},
                        {"time": "14:45", "value": 2.1},
                        {"time": "15:00", "value": 2.5}
                    ]
                },
                "throughput": {
                    "type": "sparkline",
                    "color": "#22c55e",
                    "data": [1100, 1150, 1200, 1180, 1250]
                }
            },
            "gauges": [
                {"name": "CPU", "value": 78, "max": 100, "thresholds": [{"value": 70, "color": "#f59e0b"}, {"value": 90, "color": "#ef4444"}]},
                {"name": "Memory", "value": 70, "max": 100, "thresholds": [{"value": 60, "color": "#f59e0b"}, {"value": 85, "color": "#ef4444"}]},
                {"name": "Connections", "value": 95, "max": 100, "thresholds": [{"value": 80, "color": "#f59e0b"}, {"value": 95, "color": "#ef4444"}]}
            ],
            "alerts": [
                {"severity": "critical", "badge_color": "#ef4444", "message": "P99 latency exceeded 500ms threshold", "icon": "alert-circle"},
                {"severity": "warning", "badge_color": "#f59e0b", "message": "Error rate above 2% for 15 minutes", "icon": "alert-triangle"}
            ]
        },
        "inventory-service": {
            "status": "healthy",
            "status_color": "#22c55e",  # green
            "health_score": 98,
            "metrics": [
                {"name": "Request Rate", "key": "request_rate", "value": 800, "unit": "req/min", "trend": "stable", "change": 2, "status": "normal", "icon": "activity"},
                {"name": "Error Rate", "key": "error_rate", "value": 0.1, "unit": "%", "trend": "stable", "change": 0, "status": "normal", "icon": "alert-triangle", "threshold": 1.0},
                {"name": "P50 Latency", "key": "p50_latency", "value": 25, "unit": "ms", "trend": "down", "change": -5, "status": "normal", "icon": "clock"},
                {"name": "P99 Latency", "key": "p99_latency", "value": 180, "unit": "ms", "trend": "stable", "change": -2, "status": "normal", "icon": "clock", "threshold": 500},
                {"name": "CPU Usage", "key": "cpu_usage", "value": 35, "unit": "%", "trend": "stable", "change": 1, "status": "normal", "icon": "cpu", "threshold": 80},
                {"name": "Memory", "key": "memory_usage", "value": 1.8, "unit": "GB", "trend": "stable", "change": 0.05, "status": "normal", "icon": "hard-drive", "max": 4}
            ],
            "charts": {
                "latency": {
                    "type": "area",
                    "title": "P99 Latency Over Time",
                    "color": "#22c55e",
                    "gradient": True,
                    "threshold_line": 500,
                    "y_axis": {"label": "Latency (ms)", "min": 0, "max": 500},
                    "data": [
                        {"time": "14:00", "value": 175, "baseline": 180},
                        {"time": "14:15", "value": 178, "baseline": 180},
                        {"time": "14:30", "value": 182, "baseline": 180},
                        {"time": "14:45", "value": 179, "baseline": 180},
                        {"time": "15:00", "value": 180, "baseline": 180}
                    ]
                },
                "throughput": {
                    "type": "sparkline",
                    "color": "#22c55e",
                    "data": [790, 795, 800, 805, 800]
                }
            },
            "gauges": [
                {"name": "CPU", "value": 35, "max": 100, "thresholds": [{"value": 70, "color": "#f59e0b"}, {"value": 90, "color": "#ef4444"}]},
                {"name": "Memory", "value": 45, "max": 100, "thresholds": [{"value": 60, "color": "#f59e0b"}, {"value": 85, "color": "#ef4444"}]}
            ],
            "alerts": []
        }
    }
    
    service = services.get(service_name.lower())
    if not service:
        return {"error": f"Service '{service_name}' not found", "available_services": list(services.keys())}
    
    return {
        "_ui": {"component": "ServiceDashboard", "layout": "grid"},
        "service": service_name,
        "time_range": time_range,
        "queried_at": datetime.now().isoformat(),
        **service
    }

print("✓ query_service_metrics tool defined")

In [ ]:
# Tool 2: Get active alerts - returns data for alert cards and tables
@tool
def get_active_alerts(
    severity_filter: Annotated[str, Field(description="Filter by severity: 'all', 'critical', 'warning', or 'info'")] = "all"
) -> Dict[str, Any]:
    """
    Get all active alerts with rich UI metadata for rendering alert cards,
    severity badges, and interactive tables.
    """
    alerts = [
        {
            "id": "ALT-001",
            "severity": "critical",
            "severity_badge": {"color": "#ef4444", "bg": "#fef2f2", "icon": "alert-circle"},
            "title": "High P99 Latency - Payment Service",
            "resource": {"name": "payment-service", "type": "AKS Deployment", "icon": "box", "link": "/resources/payment-service"},
            "metric": {
                "name": "p99_latency",
                "current": {"value": 892, "unit": "ms"},
                "threshold": {"value": 500, "unit": "ms"},
                "sparkline": [250, 280, 450, 720, 892],
                "sparkline_color": "#ef4444"
            },
            "timing": {
                "triggered_at": (datetime.now() - timedelta(minutes=18)).isoformat(),
                "duration": "18 minutes",
                "duration_seconds": 1080
            },
            "impact": {"level": "high", "affected_users": "~2,400", "affected_transactions": 156},
            "possible_causes": [
                {"cause": "Database connection pool exhaustion", "confidence": 0.85, "icon": "database"},
                {"cause": "Memory leak causing GC pauses", "confidence": 0.62, "icon": "cpu"},
                {"cause": "Downstream service degradation", "confidence": 0.45, "icon": "git-branch"}
            ],
            "actions": [
                {"label": "Investigate", "action": "investigate_incident", "params": {"alert_id": "ALT-001"}, "variant": "default", "icon": "search"},
                {"label": "Run Diagnostics", "action": "execute_runbook", "params": {"runbook_name": "diagnostics", "target_resource": "payment-service"}, "variant": "outline", "icon": "terminal"}
            ]
        },
        {
            "id": "ALT-002",
            "severity": "critical",
            "severity_badge": {"color": "#ef4444", "bg": "#fef2f2", "icon": "alert-circle"},
            "title": "Pod Restart Loop - Payment Service",
            "resource": {"name": "payment-service-7d4f8b9c5-x2k9j", "type": "AKS Pod", "icon": "box", "link": "/resources/pods/payment-service-7d4f8b9c5-x2k9j"},
            "metric": {
                "name": "restart_count",
                "current": {"value": 8, "unit": "restarts"},
                "threshold": {"value": 3, "unit": "restarts/hour"},
                "sparkline": [0, 1, 2, 5, 8],
                "sparkline_color": "#ef4444"
            },
            "timing": {
                "triggered_at": (datetime.now() - timedelta(minutes=12)).isoformat(),
                "duration": "12 minutes",
                "duration_seconds": 720
            },
            "impact": {"level": "high", "affected_users": "~1,800", "affected_transactions": 89},
            "possible_causes": [
                {"cause": "OOMKilled - Memory limit exceeded", "confidence": 0.91, "icon": "hard-drive"},
                {"cause": "Liveness probe failures", "confidence": 0.45, "icon": "heart-pulse"},
                {"cause": "Application crash on startup", "confidence": 0.32, "icon": "x-circle"}
            ],
            "actions": [
                {"label": "View Logs", "action": "view_logs", "params": {"pod": "payment-service-7d4f8b9c5-x2k9j"}, "variant": "default", "icon": "file-text"},
                {"label": "Restart Pod", "action": "execute_runbook", "params": {"runbook_name": "restart-service", "target_resource": "payment-service"}, "variant": "destructive", "icon": "refresh-cw"}
            ]
        },
        {
            "id": "ALT-003",
            "severity": "warning",
            "severity_badge": {"color": "#f59e0b", "bg": "#fffbeb", "icon": "alert-triangle"},
            "title": "Elevated Error Rate - Payment Service",
            "resource": {"name": "payment-service", "type": "AKS Deployment", "icon": "box"},
            "metric": {
                "name": "error_rate",
                "current": {"value": 2.5, "unit": "%"},
                "threshold": {"value": 1, "unit": "%"},
                "sparkline": [0.5, 0.7, 1.2, 2.1, 2.5],
                "sparkline_color": "#f59e0b"
            },
            "timing": {
                "triggered_at": (datetime.now() - timedelta(minutes=15)).isoformat(),
                "duration": "15 minutes",
                "duration_seconds": 900
            },
            "impact": {"level": "medium", "affected_users": "~600", "affected_transactions": 45},
            "possible_causes": [
                {"cause": "Upstream dependency failures", "confidence": 0.72, "icon": "git-branch"},
                {"cause": "Invalid request payload", "confidence": 0.55, "icon": "file-x"},
                {"cause": "Configuration drift", "confidence": 0.38, "icon": "settings"}
            ],
            "actions": [
                {"label": "View Errors", "action": "view_errors", "variant": "default", "icon": "bug"},
                {"label": "Acknowledge", "action": "acknowledge_alert", "variant": "outline", "icon": "check"}
            ]
        },
        {
            "id": "ALT-004",
            "severity": "warning",
            "severity_badge": {"color": "#f59e0b", "bg": "#fffbeb", "icon": "alert-triangle"},
            "title": "High CPU Usage - Database Server",
            "resource": {"name": "sql-prod-001", "type": "Azure SQL", "icon": "database"},
            "metric": {
                "name": "cpu_percent",
                "current": {"value": 78, "unit": "%"},
                "threshold": {"value": 70, "unit": "%"},
                "sparkline": [55, 62, 68, 74, 78],
                "sparkline_color": "#f59e0b"
            },
            "timing": {
                "triggered_at": (datetime.now() - timedelta(minutes=25)).isoformat(),
                "duration": "25 minutes",
                "duration_seconds": 1500
            },
            "impact": {"level": "medium", "affected_users": "All", "affected_transactions": None},
            "possible_causes": [
                {"cause": "Expensive queries from payment service", "confidence": 0.88, "icon": "search"},
                {"cause": "Missing index on frequently queried table", "confidence": 0.75, "icon": "list"},
                {"cause": "Lock contention during peak load", "confidence": 0.52, "icon": "lock"}
            ],
            "actions": [
                {"label": "Query Insights", "action": "view_query_insights", "variant": "default", "icon": "bar-chart"},
                {"label": "Scale Up", "action": "scale_database", "variant": "outline", "icon": "trending-up"}
            ]
        }
    ]
    
    if severity_filter != "all":
        alerts = [a for a in alerts if a["severity"] == severity_filter]
    
    # Summary with visual stats
    all_alerts = alerts if severity_filter == "all" else [
        {"severity": "critical"}, {"severity": "critical"},
        {"severity": "warning"}, {"severity": "warning"}
    ]
    summary = {
        "total": len(alerts),
        "stats": [
            {"label": "Critical", "value": len([a for a in all_alerts if a["severity"] == "critical"]), "color": "#ef4444", "icon": "alert-circle"},
            {"label": "Warning", "value": len([a for a in all_alerts if a["severity"] == "warning"]), "color": "#f59e0b", "icon": "alert-triangle"},
            {"label": "Info", "value": len([a for a in all_alerts if a.get("severity") == "info"]), "color": "#3b82f6", "icon": "info"}
        ],
        "trend": {"direction": "up", "change": 2, "period": "vs last hour"}
    }
    
    return {
        "_ui": {"component": "AlertDashboard", "layout": "list"},
        "summary": summary,
        "alerts": alerts,
        "queried_at": datetime.now().isoformat()
    }

print("✓ get_active_alerts tool defined")

In [ ]:
# Tool 3: Execute runbook - returns step-by-step progress data
@tool
def execute_runbook(
    runbook_name: Annotated[str, Field(description="Name of the runbook to execute")],
    target_resource: Annotated[str, Field(description="Resource to apply the runbook to")],
    dry_run: Annotated[bool, Field(description="If true, simulate execution without making changes")] = True
) -> Dict[str, Any]:
    """
    Execute an automation runbook with rich UI progress tracking.
    Returns data for stepper components, progress bars, and status badges.
    IMPORTANT: Always use dry_run=true first to preview changes.
    """
    runbooks = {
        "restart-service": {
            "title": "Restart Service",
            "description": "Gracefully restart a service with health checks",
            "icon": "refresh-cw",
            "risk": {"level": "low", "color": "#22c55e", "badge": "Low Risk"},
            "estimated_duration": {"min": 2, "max": 3, "unit": "minutes"},
            "steps": [
                {"id": 1, "title": "Scale Down", "description": "Scale deployment to 0 replicas", "duration": "30s", "icon": "arrow-down"},
                {"id": 2, "title": "Graceful Shutdown", "description": "Wait for graceful shutdown", "duration": "15s", "icon": "clock"},
                {"id": 3, "title": "Clear Caches", "description": "Clear local caches", "duration": "5s", "icon": "trash-2"},
                {"id": 4, "title": "Scale Up", "description": "Scale deployment back to original count", "duration": "30s", "icon": "arrow-up"},
                {"id": 5, "title": "Wait Ready", "description": "Wait for pods to be ready", "duration": "60s", "icon": "loader"},
                {"id": 6, "title": "Health Check", "description": "Run health check validation", "duration": "15s", "icon": "heart-pulse"}
            ],
            "rollback_available": True
        },
        "scale-out": {
            "title": "Scale Out",
            "description": "Increase replica count to handle load",
            "icon": "maximize-2",
            "risk": {"level": "low", "color": "#22c55e", "badge": "Low Risk"},
            "estimated_duration": {"min": 1, "max": 2, "unit": "minutes"},
            "steps": [
                {"id": 1, "title": "Check Replicas", "description": "Check current replica count", "duration": "5s", "icon": "hash"},
                {"id": 2, "title": "Verify Resources", "description": "Verify cluster has sufficient resources", "duration": "10s", "icon": "cpu"},
                {"id": 3, "title": "Scale", "description": "Increase replica count by 50%", "duration": "30s", "icon": "plus-circle"},
                {"id": 4, "title": "Schedule Pods", "description": "Wait for new pods to be scheduled", "duration": "45s", "icon": "loader"},
                {"id": 5, "title": "Register LB", "description": "Verify load balancer registration", "duration": "15s", "icon": "globe"}
            ],
            "rollback_available": True
        },
        "clear-connection-pool": {
            "title": "Clear Connection Pool",
            "description": "Reset database connection pool to clear stale connections",
            "icon": "database",
            "risk": {"level": "medium", "color": "#f59e0b", "badge": "Medium Risk"},
            "estimated_duration": {"min": 0.5, "max": 1, "unit": "minutes"},
            "steps": [
                {"id": 1, "title": "Identify Pods", "description": "Identify pods with high connection age", "duration": "10s", "icon": "search"},
                {"id": 2, "title": "Refresh Pool", "description": "Trigger connection pool refresh", "duration": "5s", "icon": "refresh-cw"},
                {"id": 3, "title": "Verify", "description": "Verify new connections established", "duration": "10s", "icon": "check-circle"},
                {"id": 4, "title": "Monitor", "description": "Monitor query latency for improvement", "duration": "30s", "icon": "activity"}
            ],
            "rollback_available": False
        },
        "rollback-deployment": {
            "title": "Rollback Deployment",
            "description": "Roll back to the previous stable deployment",
            "icon": "rotate-ccw",
            "risk": {"level": "medium", "color": "#f59e0b", "badge": "Medium Risk"},
            "estimated_duration": {"min": 3, "max": 5, "unit": "minutes"},
            "steps": [
                {"id": 1, "title": "Find Revision", "description": "Identify previous stable revision", "duration": "10s", "icon": "git-commit"},
                {"id": 2, "title": "Verify Image", "description": "Verify previous image is available", "duration": "15s", "icon": "package"},
                {"id": 3, "title": "Rolling Update", "description": "Initiate rolling update to previous version", "duration": "60s", "icon": "refresh-cw"},
                {"id": 4, "title": "Monitor Rollout", "description": "Monitor rollout progress", "duration": "90s", "icon": "activity"},
                {"id": 5, "title": "Smoke Tests", "description": "Run smoke tests", "duration": "30s", "icon": "check-square"},
                {"id": 6, "title": "Update Annotations", "description": "Update deployment annotations", "duration": "5s", "icon": "edit"}
            ],
            "rollback_available": True
        }
    }
    
    runbook = runbooks.get(runbook_name)
    if not runbook:
        return {
            "_ui": {"component": "RunbookSelector"},
            "error": f"Runbook '{runbook_name}' not found",
            "available_runbooks": [
                {"name": name, "title": rb["title"], "description": rb["description"], "icon": rb["icon"], "risk": rb["risk"]}
                for name, rb in runbooks.items()
            ]
        }
    
    # Build step statuses
    if dry_run:
        steps = [{**step, "status": "pending", "status_icon": "circle"} for step in runbook["steps"]]
        progress = 0
        status = {"state": "preview", "color": "#3b82f6", "icon": "eye", "label": "Preview Mode"}
    else:
        steps = [{**step, "status": "completed", "status_icon": "check-circle", "output": "Success"} for step in runbook["steps"]]
        progress = 100
        status = {"state": "completed", "color": "#22c55e", "icon": "check-circle", "label": "Completed"}
    
    return {
        "_ui": {"component": "RunbookExecution", "layout": "stepper"},
        "runbook": {
            "name": runbook_name,
            "title": runbook["title"],
            "description": runbook["description"],
            "icon": runbook["icon"]
        },
        "target_resource": target_resource,
        "status": status,
        "risk": runbook["risk"],
        "estimated_duration": runbook["estimated_duration"],
        "rollback_available": runbook["rollback_available"],
        "progress": {"value": progress, "max": 100},
        "steps": steps,
        "execution": {
            "id": f"exec-{datetime.now().strftime('%Y%m%d%H%M%S')}" if not dry_run else None,
            "started_at": datetime.now().isoformat(),
            "completed_at": (datetime.now() + timedelta(minutes=2)).isoformat() if not dry_run else None
        },
        "actions": [
            {"label": "Execute", "action": "execute", "variant": "default", "icon": "play", "disabled": not dry_run},
            {"label": "Cancel", "action": "cancel", "variant": "outline", "icon": "x"}
        ] if dry_run else [
            {"label": "View Logs", "action": "view_logs", "variant": "outline", "icon": "file-text"},
            {"label": "Rollback", "action": "rollback", "variant": "destructive", "icon": "rotate-ccw", "disabled": not runbook["rollback_available"]}
        ]
    }

print("✓ execute_runbook tool defined")

In [ ]:
# Tool 4: Investigate incident - returns timeline and analysis data
@tool
def investigate_incident(
    alert_id: Annotated[str, Field(description="Alert ID to investigate (e.g., 'ALT-001')")]
) -> Dict[str, Any]:
    """
    Perform root cause analysis with rich visual data including timelines,
    confidence meters, correlation graphs, and recommended actions.
    """
    investigations = {
        "ALT-001": {
            "alert_id": "ALT-001",
            "title": "High P99 Latency - Payment Service",
            "status": {"state": "completed", "color": "#22c55e", "icon": "check-circle"},
            "summary_card": {
                "headline": "Connection Pool Exhaustion",
                "subtext": "Caused by deployment v2.3.1 reducing pool size",
                "confidence": 92,
                "confidence_color": "#22c55e",
                "icon": "database"
            },
            "timeline": {
                "title": "Event Timeline",
                "events": [
                    {"time": "14:28:00", "event": "Deployment payment-service:v2.3.1 completed", "source": "ArgoCD", "icon": "git-commit", "type": "deployment", "color": "#3b82f6"},
                    {"time": "14:32:15", "event": "First slow query detected (>500ms)", "source": "APM", "icon": "clock", "type": "anomaly", "color": "#f59e0b"},
                    {"time": "14:35:00", "event": "Connection pool utilization hit 95%", "source": "Prometheus", "icon": "trending-up", "type": "metric", "color": "#f59e0b"},
                    {"time": "14:38:45", "event": "P99 latency crossed 500ms threshold", "source": "Grafana", "icon": "alert-triangle", "type": "threshold", "color": "#ef4444"},
                    {"time": "14:42:00", "event": "First pod restart due to OOMKilled", "source": "Kubernetes", "icon": "x-circle", "type": "error", "color": "#ef4444"},
                    {"time": "14:45:00", "event": "Alert ALT-001 triggered", "source": "AlertManager", "icon": "bell", "type": "alert", "color": "#ef4444"}
                ]
            },
            "correlated_events": [
                {"type": "deployment", "icon": "git-commit", "color": "#3b82f6", "title": "Recent Deployment", "description": "payment-service:v2.3.1 deployed 17 minutes ago", "relevance": 0.95},
                {"type": "config_change", "icon": "settings", "color": "#f59e0b", "title": "Configuration Change", "description": "Connection pool size reduced from 100 to 50 in v2.3.1", "relevance": 0.98},
                {"type": "traffic", "icon": "trending-up", "color": "#22c55e", "title": "Traffic Pattern", "description": "Traffic increased 15% (normal for this time)", "relevance": 0.45}
            ],
            "log_analysis": {
                "total_errors": 1247,
                "time_range": "Last 30 minutes",
                "patterns": [
                    {"pattern": "Connection pool exhausted", "count": 892, "percentage": 71.5, "color": "#ef4444", "severity": "critical"},
                    {"pattern": "Query timeout after 30000ms", "count": 245, "percentage": 19.6, "color": "#f59e0b", "severity": "warning"},
                    {"pattern": "OutOfMemoryError: heap space", "count": 110, "percentage": 8.8, "color": "#f59e0b", "severity": "warning"}
                ],
                "chart": {
                    "type": "donut",
                    "data": [{"name": "Connection pool", "value": 71.5, "color": "#ef4444"}, {"name": "Timeout", "value": 19.6, "color": "#f59e0b"}, {"name": "OOM", "value": 8.8, "color": "#fbbf24"}]
                }
            },
            "root_cause": {
                "confidence": 92,
                "confidence_level": "high",
                "confidence_color": "#22c55e",
                "primary_cause": "Database connection pool exhaustion",
                "explanation": "Deployment v2.3.1 reduced connection pool size from 100 to 50. Combined with normal traffic increase, this caused connection starvation, leading to query timeouts and cascading failures.",
                "evidence": [
                    {"fact": "Connection pool size reduced by 50%", "confidence": 98, "icon": "database"},
                    {"fact": "71.5% of errors are pool exhaustion", "confidence": 95, "icon": "pie-chart"},
                    {"fact": "Issue started 4 min after deployment", "confidence": 88, "icon": "clock"}
                ],
                "contributing_factors": [
                    {"factor": "Connection pool size reduced by 50%", "impact": "high", "icon": "settings"},
                    {"factor": "No connection pool monitoring alerts", "impact": "medium", "icon": "bell-off"},
                    {"factor": "Traffic increased 15%", "impact": "low", "icon": "trending-up"}
                ]
            },
            "recommended_actions": [
                {
                    "priority": 1,
                    "priority_badge": {"label": "P1", "color": "#ef4444"},
                    "title": "Clear Connection Pool",
                    "description": "Execute 'clear-connection-pool' runbook for immediate relief",
                    "impact": "Immediate relief - clears stale connections",
                    "risk": {"level": "low", "color": "#22c55e"},
                    "action": {"type": "runbook", "name": "clear-connection-pool", "target": "payment-service"},
                    "button": {"label": "Execute Runbook", "variant": "default", "icon": "play"}
                },
                {
                    "priority": 2,
                    "priority_badge": {"label": "P2", "color": "#f59e0b"},
                    "title": "Rollback Deployment",
                    "description": "Rollback to payment-service:v2.3.0",
                    "impact": "Restores original connection pool settings",
                    "risk": {"level": "medium", "color": "#f59e0b"},
                    "action": {"type": "runbook", "name": "rollback-deployment", "target": "payment-service"},
                    "button": {"label": "Rollback", "variant": "outline", "icon": "rotate-ccw"}
                },
                {
                    "priority": 3,
                    "priority_badge": {"label": "P3", "color": "#3b82f6"},
                    "title": "Scale Out",
                    "description": "Scale payment-service to 6 replicas",
                    "impact": "Distributes load across more instances",
                    "risk": {"level": "low", "color": "#22c55e"},
                    "action": {"type": "runbook", "name": "scale-out", "target": "payment-service"},
                    "button": {"label": "Scale Out", "variant": "outline", "icon": "maximize-2"}
                }
            ],
            "similar_incidents": [
                {"id": "INC-2024-0115", "similarity": 87, "resolution": "Increased connection pool size", "date": "2024-01-15", "link": "/incidents/INC-2024-0115"},
                {"id": "INC-2023-1203", "similarity": 72, "resolution": "Rolled back deployment", "date": "2023-12-03", "link": "/incidents/INC-2023-1203"}
            ]
        }
    }
    
    investigation = investigations.get(alert_id)
    if not investigation:
        return {
            "_ui": {"component": "ErrorMessage"},
            "error": f"No investigation data for alert '{alert_id}'",
            "suggestion": "Use get_active_alerts() to see current alerts, then investigate one."
        }
    
    return {
        "_ui": {"component": "InvestigationReport", "layout": "sections"},
        **investigation
    }

print("✓ investigate_incident tool defined")

In [ ]:
# Tool 5: Send notification with preview
@tool  
def send_notification(
    channel: Annotated[str, Field(description="Notification channel: 'teams', 'slack', 'pagerduty', or 'email'")],
    message: Annotated[str, Field(description="Message content (markdown supported)")],
    severity: Annotated[str, Field(description="Severity level: 'info', 'warning', or 'critical'")] = "info",
    recipients: Annotated[str, Field(description="Recipients or channel name")] = "sre-oncall"
) -> Dict[str, Any]:
    """
    Send a notification with a rich preview showing how it will appear.
    """
    channel_config = {
        "teams": {"icon": "message-square", "color": "#5059c9", "name": "Microsoft Teams"},
        "slack": {"icon": "hash", "color": "#4a154b", "name": "Slack"},
        "pagerduty": {"icon": "phone-call", "color": "#06ac38", "name": "PagerDuty"},
        "email": {"icon": "mail", "color": "#ea4335", "name": "Email"}
    }
    
    severity_config = {
        "info": {"color": "#3b82f6", "bg": "#eff6ff", "icon": "info", "label": "Info"},
        "warning": {"color": "#f59e0b", "bg": "#fffbeb", "icon": "alert-triangle", "label": "Warning"},
        "critical": {"color": "#ef4444", "bg": "#fef2f2", "icon": "alert-circle", "label": "Critical"}
    }
    
    ch = channel_config.get(channel.lower(), channel_config["teams"])
    sev = severity_config.get(severity, severity_config["info"])
    
    return {
        "_ui": {"component": "NotificationConfirmation"},
        "status": {"state": "sent", "color": "#22c55e", "icon": "check-circle", "label": "Sent Successfully"},
        "notification": {
            "channel": {
                "type": channel,
                "name": ch["name"],
                "icon": ch["icon"],
                "color": ch["color"]
            },
            "recipients": recipients,
            "severity": {
                "level": severity,
                **sev
            },
            "preview": {
                "card_color": sev["color"],
                "message": message,
                "formatted": True
            }
        },
        "delivery": {
            "sent_at": datetime.now().isoformat(),
            "message_id": f"msg-{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "estimated_delivery": "< 1 second"
        },
        "actions": [
            {"label": "View in " + ch["name"], "action": "open_channel", "variant": "outline", "icon": "external-link"},
            {"label": "Send Another", "action": "new_notification", "variant": "ghost", "icon": "plus"}
        ]
    }

print("✓ send_notification tool defined")
print("\n✓ All 5 SRE tools defined with rich UI metadata!")

---

## Step 3: Create the SRE Agent

In [ ]:
# Configuration
PROJECT_ENDPOINT = os.environ.get("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

if not PROJECT_ENDPOINT or PROJECT_ENDPOINT.startswith("https://<"):
    print("⚠️  PROJECT_ENDPOINT not configured in .env file")
    print("   Copy .env.sample to .env and set your Azure AI Foundry endpoint")
    raise ValueError("PROJECT_ENDPOINT not configured")
else:
    print(f"✓ Using endpoint: {PROJECT_ENDPOINT[:50]}...")
    print(f"✓ Using model: {MODEL_DEPLOYMENT}")

In [ ]:
SRE_INSTRUCTIONS = """
You are an Azure SRE Agent assistant. Your role is to help Site Reliability Engineers 
monitor, diagnose, and remediate issues in their Azure infrastructure.

## Your Capabilities
1. **Monitoring**: Query service metrics and check system health
2. **Alerting**: View and triage active alerts across the infrastructure  
3. **Investigation**: Perform root cause analysis on incidents
4. **Remediation**: Execute runbooks to fix issues (always dry-run first!)
5. **Communication**: Send notifications to the team

## Guidelines
- Always check current alerts and metrics before suggesting actions
- When executing runbooks, ALWAYS do a dry-run first to preview changes
- Provide clear explanations of what you find and recommend
- For critical issues, suggest notifying the on-call team
"""

# Initialize credential and client
credential = DefaultAzureCredential()

chat_client = AzureAIAgentClient(
    project_endpoint=PROJECT_ENDPOINT,
    credential=credential,
    model_deployment_name=MODEL_DEPLOYMENT,
)

# Create the agent with all SRE tools
sre_agent = ChatAgent(
    name="SREAgent",
    instructions=SRE_INSTRUCTIONS,
    chat_client=chat_client,
    tools=[query_service_metrics, get_active_alerts, execute_runbook, investigate_incident, send_notification],
)

print("✓ SRE Agent created with 5 tools")

---

## Step 4: Test the Agent Locally

In [ ]:
# Test the agent
thread = sre_agent.get_new_thread()

print("Query: 'What alerts are currently active?'")
print("=" * 60)

async for update in sre_agent.run_stream("What alerts are currently active?", thread=thread):
    if update.text:
        print(update.text, end="", flush=True)

print("\n")

---

## Step 5: Set Up AG-UI Server

In [ ]:
from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI(title="SRE Agent - AG-UI Server", version="1.0.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

add_agent_framework_fastapi_endpoint(app, sre_agent, "/")

print("✓ AG-UI endpoint registered at POST /")

---

## Step 6: Run the AG-UI Server

In [ ]:
import uvicorn
import threading
import asyncio

# Create a new agent and server in a separate thread with its own event loop
def run_server():
    # Create a new event loop for this thread
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    # Create fresh async credential and client in this thread's loop
    from azure.identity.aio import DefaultAzureCredential as AsyncDefaultAzureCredential
    
    credential = AsyncDefaultAzureCredential()
    
    chat_client = AzureAIAgentClient(
        project_endpoint=PROJECT_ENDPOINT,
        credential=credential,
        model_deployment_name=MODEL_DEPLOYMENT,
    )
    
    # Create fresh agent
    server_agent = ChatAgent(
        name="SREAgent",
        instructions=SRE_INSTRUCTIONS,
        chat_client=chat_client,
        tools=[query_service_metrics, get_active_alerts, execute_runbook, investigate_incident, send_notification],
    )
    
    # Create fresh FastAPI app
    from fastapi import FastAPI
    from fastapi.middleware.cors import CORSMiddleware
    from agent_framework_ag_ui import add_agent_framework_fastapi_endpoint
    
    server_app = FastAPI(title="SRE Agent - AG-UI Server", version="1.0.0")
    server_app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )
    add_agent_framework_fastapi_endpoint(server_app, server_agent, "/")
    
    # Run uvicorn
    uvicorn.run(server_app, host="127.0.0.1", port=8888, log_level="info")

# Start server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

import time
time.sleep(2)  # Give server time to start

print("🚀 AG-UI Server started at http://127.0.0.1:8888")
print("")
print("Connect with:")
print("  • CopilotKit: Set agent URL to http://127.0.0.1:8888")
print("  • CLI: python agui_client.py")
print("")
print("Server running in background. Restart kernel to stop.")

---

## Step 7: Test the AG-UI Endpoint

Send a request to the AG-UI server and see the raw streaming response with `_ui` hints.

In [ ]:
import httpx
import json

# Test the AG-UI endpoint with streaming
print("Sending request to AG-UI server...")
print("Query: 'Show me current alerts'")
print("=" * 60)

text_buffer = ""

with httpx.stream(
    "POST",
    "http://127.0.0.1:8888",
    json={
        "messages": [{"role": "user", "content": "Show me current alerts"}],
        "runId": "test-run-001"
    },
    headers={"Content-Type": "application/json", "Accept": "text/event-stream"},
    timeout=60.0
) as response:
    for line in response.iter_lines():
        if line.startswith("data:"):
            data = line[5:].strip()
            if data and data != "[DONE]":
                try:
                    event = json.loads(data)
                    event_type = event.get("type", "unknown")
                    
                    if event_type == "TEXT_MESSAGE_CONTENT":
                        # Accumulate text, print on newlines for readability
                        delta = event.get("delta", "")
                        text_buffer += delta
                        # Print accumulated text at natural breaks
                        if "\n" in delta:
                            print(text_buffer.rstrip(), flush=True)
                            text_buffer = ""
                    elif event_type == "TOOL_CALL_START":
                        if text_buffer:
                            print(text_buffer.rstrip())
                            text_buffer = ""
                        print(f"\n🔧 Tool Call: {event.get('toolCallName', 'unknown')}")
                    elif event_type == "TOOL_CALL_END":
                        result = event.get("toolCallResult", "{}")
                        try:
                            parsed = json.loads(result) if isinstance(result, str) else result
                            if "_ui" in parsed:
                                print(f"   🎨 UI Hint: {parsed['_ui']}")
                            if "summary" in parsed:
                                stats = parsed['summary'].get('stats', [])
                                print(f"   📊 Summary: {parsed['summary']['total']} alerts")
                                for s in stats:
                                    if s['value'] > 0:
                                        print(f"      • {s['label']}: {s['value']}")
                            if "alerts" in parsed:
                                print(f"   🚨 Found {len(parsed['alerts'])} alerts")
                        except:
                            pass
                    elif event_type == "RUN_STARTED":
                        print(f"▶️ Run started")
                    elif event_type == "RUN_FINISHED":
                        if text_buffer:
                            print(text_buffer.rstrip())
                            text_buffer = ""
                        print(f"\n✅ Run finished")
                except json.JSONDecodeError:
                    pass

# Print any remaining text
if text_buffer:
    print(text_buffer.rstrip())

print("\n" + "=" * 60)

---

## Step 8: Rich UI Visualization (In-Notebook)

Render the tool output data as visual components using HTML/CSS directly in the notebook. This shows how a frontend would interpret the `_ui` hints.

In [ ]:
from IPython.display import HTML, display

# Get alerts data directly from the tool
alerts_data = get_active_alerts(severity_filter="all")

# Build HTML dashboard
html = """
<style>
    .sre-dashboard { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif; max-width: 900px; }
    .stat-cards { display: flex; gap: 12px; margin-bottom: 20px; }
    .stat-card { 
        flex: 1; padding: 16px; border-radius: 8px; text-align: center;
        background: linear-gradient(135deg, #f8fafc 0%, #f1f5f9 100%);
        border: 1px solid #e2e8f0;
    }
    .stat-value { font-size: 32px; font-weight: 700; }
    .stat-label { font-size: 12px; color: #64748b; text-transform: uppercase; letter-spacing: 0.5px; }
    .alert-card {
        background: white; border-radius: 8px; padding: 16px; margin-bottom: 12px;
        border-left: 4px solid; box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }
    .alert-critical { border-left-color: #ef4444; background: #fef2f2; }
    .alert-warning { border-left-color: #f59e0b; background: #fffbeb; }
    .alert-header { display: flex; justify-content: space-between; align-items: center; margin-bottom: 8px; }
    .alert-title { font-weight: 600; font-size: 14px; color: #1e293b; }
    .badge { 
        padding: 2px 8px; border-radius: 9999px; font-size: 11px; font-weight: 600;
        text-transform: uppercase;
    }
    .badge-critical { background: #fef2f2; color: #dc2626; }
    .badge-warning { background: #fffbeb; color: #d97706; }
    .alert-meta { font-size: 12px; color: #64748b; }
    .alert-metric { 
        display: inline-flex; align-items: center; gap: 4px; 
        background: #f1f5f9; padding: 4px 8px; border-radius: 4px; margin-top: 8px;
        font-family: monospace; font-size: 13px;
    }
    .sparkline { display: inline-block; height: 20px; margin-left: 8px; }
    .sparkline-bar { 
        display: inline-block; width: 6px; margin-right: 2px; 
        border-radius: 2px; vertical-align: bottom;
    }
    .causes { margin-top: 12px; }
    .cause { 
        display: flex; align-items: center; gap: 8px; padding: 6px 0;
        font-size: 13px; color: #475569;
    }
    .confidence-bar { 
        width: 60px; height: 6px; background: #e2e8f0; border-radius: 3px; overflow: hidden;
    }
    .confidence-fill { height: 100%; border-radius: 3px; }
    .section-title { font-size: 18px; font-weight: 600; margin: 20px 0 12px; color: #1e293b; }
</style>

<div class="sre-dashboard">
    <h2 style="margin-bottom: 20px;">🚨 Active Alerts Dashboard</h2>
"""

# Add stat cards
summary = alerts_data["summary"]
html += '<div class="stat-cards">'
for stat in summary["stats"]:
    html += f'''
    <div class="stat-card">
        <div class="stat-value" style="color: {stat['color']}">{stat['value']}</div>
        <div class="stat-label">{stat['label']}</div>
    </div>
    '''
html += '</div>'

# Add alert cards
html += '<div class="section-title">Alert Details</div>'
for alert in alerts_data["alerts"]:
    severity = alert["severity"]
    badge_class = f"badge-{severity}"
    card_class = f"alert-{severity}"
    
    # Build sparkline
    sparkline_data = alert["metric"].get("sparkline", [])
    max_val = max(sparkline_data) if sparkline_data else 1
    sparkline_html = '<span class="sparkline">'
    for val in sparkline_data:
        height = int((val / max_val) * 20)
        color = alert["metric"].get("sparkline_color", "#94a3b8")
        sparkline_html += f'<span class="sparkline-bar" style="height: {height}px; background: {color};"></span>'
    sparkline_html += '</span>'
    
    # Build causes
    causes_html = '<div class="causes">'
    for cause in alert.get("possible_causes", [])[:2]:
        conf = int(cause["confidence"] * 100)
        conf_color = "#22c55e" if conf > 70 else "#f59e0b" if conf > 50 else "#94a3b8"
        causes_html += f'''
        <div class="cause">
            <div class="confidence-bar"><div class="confidence-fill" style="width: {conf}%; background: {conf_color};"></div></div>
            <span>{conf}%</span>
            <span>{cause['cause']}</span>
        </div>
        '''
    causes_html += '</div>'
    
    metric = alert["metric"]
    html += f'''
    <div class="alert-card {card_class}">
        <div class="alert-header">
            <span class="alert-title">{alert['title']}</span>
            <span class="badge {badge_class}">{severity}</span>
        </div>
        <div class="alert-meta">
            📦 {alert['resource']['name']} • ⏱️ {alert['timing']['duration']}
        </div>
        <div class="alert-metric">
            {metric['name']}: <strong>{metric['current']['value']}{metric['current']['unit']}</strong>
            (threshold: {metric['threshold']['value']}{metric['threshold']['unit']})
            {sparkline_html}
        </div>
        {causes_html}
    </div>
    '''

html += '</div>'

display(HTML(html))

In [ ]:
# Visualize Service Metrics Dashboard
metrics_data = query_service_metrics(service_name="payment-service", time_range="1h")

html = """
<style>
    .metrics-dashboard { font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif; max-width: 900px; }
    .health-header { 
        display: flex; justify-content: space-between; align-items: center;
        padding: 16px; background: linear-gradient(135deg, #1e293b 0%, #334155 100%);
        border-radius: 8px; color: white; margin-bottom: 20px;
    }
    .health-score { font-size: 48px; font-weight: 700; }
    .health-label { font-size: 14px; opacity: 0.8; }
    .metrics-grid { display: grid; grid-template-columns: repeat(3, 1fr); gap: 12px; margin-bottom: 20px; }
    .metric-card {
        padding: 16px; border-radius: 8px; background: white;
        border: 1px solid #e2e8f0; box-shadow: 0 1px 2px rgba(0,0,0,0.05);
    }
    .metric-name { font-size: 12px; color: #64748b; margin-bottom: 4px; }
    .metric-value { font-size: 24px; font-weight: 600; }
    .metric-change { font-size: 12px; margin-top: 4px; }
    .change-up { color: #ef4444; }
    .change-down { color: #22c55e; }
    .change-stable { color: #64748b; }
    .status-normal { border-left: 3px solid #22c55e; }
    .status-warning { border-left: 3px solid #f59e0b; }
    .status-critical { border-left: 3px solid #ef4444; }
    .chart-container { 
        background: white; border-radius: 8px; padding: 16px;
        border: 1px solid #e2e8f0; margin-bottom: 16px;
    }
    .chart-title { font-weight: 600; margin-bottom: 12px; }
    .bar-chart { display: flex; align-items: flex-end; height: 100px; gap: 8px; }
    .bar { 
        flex: 1; border-radius: 4px 4px 0 0; 
        display: flex; flex-direction: column; justify-content: flex-end; align-items: center;
    }
    .bar-value { font-size: 10px; color: #64748b; margin-bottom: 4px; }
    .bar-fill { width: 100%; border-radius: 4px 4px 0 0; }
    .bar-label { font-size: 10px; color: #64748b; margin-top: 4px; }
    .gauge-container { display: flex; gap: 16px; justify-content: center; }
    .gauge { text-align: center; flex: 1; max-width: 120px; }
    .gauge-visual { 
        width: 80px; height: 40px; margin: 0 auto;
        border-radius: 80px 80px 0 0; overflow: hidden;
        background: #e2e8f0; position: relative;
    }
    .gauge-fill { 
        position: absolute; bottom: 0; left: 0; right: 0;
        border-radius: 80px 80px 0 0;
    }
    .gauge-value { font-size: 20px; font-weight: 600; margin-top: 8px; }
    .gauge-label { font-size: 12px; color: #64748b; }
</style>

<div class="metrics-dashboard">
"""

# Health header
status_color = metrics_data.get("status_color", "#22c55e")
html += f'''
<div class="health-header">
    <div>
        <div style="font-size: 20px; font-weight: 600;">📊 {metrics_data['service']}</div>
        <div style="font-size: 14px; opacity: 0.8;">Status: {metrics_data['status'].upper()}</div>
    </div>
    <div style="text-align: right;">
        <div class="health-score" style="color: {status_color};">{metrics_data['health_score']}</div>
        <div class="health-label">Health Score</div>
    </div>
</div>
'''

# Metrics grid
html += '<div class="metrics-grid">'
for metric in metrics_data["metrics"]:
    status_class = f"status-{metric['status']}"
    trend = metric.get("trend", "stable")
    change = metric.get("change", 0)
    change_class = "change-up" if trend == "up" else "change-down" if trend == "down" else "change-stable"
    arrow = "↑" if trend == "up" else "↓" if trend == "down" else "→"
    
    html += f'''
    <div class="metric-card {status_class}">
        <div class="metric-name">{metric['name']}</div>
        <div class="metric-value">{metric['value']}<span style="font-size: 14px; color: #64748b;"> {metric['unit']}</span></div>
        <div class="metric-change {change_class}">{arrow} {abs(change)} {metric['unit']}</div>
    </div>
    '''
html += '</div>'

# Bar chart for error rate
error_chart = metrics_data["charts"].get("error_rate", {})
if error_chart:
    max_val = 5
    threshold = error_chart.get("threshold_line", 1.0)
    html += f'''
    <div class="chart-container">
        <div class="chart-title">📈 {error_chart.get('title', 'Error Rate')}</div>
        <div class="bar-chart">
    '''
    for point in error_chart.get("data", []):
        height = int((point["value"] / max_val) * 80)
        color = "#ef4444" if point["value"] > threshold else "#f59e0b" if point["value"] > 0.5 else "#22c55e"
        html += f'''
        <div class="bar">
            <div class="bar-value">{point['value']}%</div>
            <div class="bar-fill" style="height: {height}px; background: {color};"></div>
            <div class="bar-label">{point['time']}</div>
        </div>
        '''
    html += '</div></div>'

# Gauges
html += '''
<div class="chart-container">
    <div class="chart-title">⚡ Resource Utilization</div>
    <div class="gauge-container">
'''
for gauge in metrics_data.get("gauges", []):
    value = gauge["value"]
    # Determine color based on thresholds
    color = "#22c55e"
    for thresh in gauge.get("thresholds", []):
        if value >= thresh["value"]:
            color = thresh["color"]
    height = int((value / gauge["max"]) * 40)
    
    html += f'''
    <div class="gauge">
        <div class="gauge-visual">
            <div class="gauge-fill" style="height: {height}px; background: {color};"></div>
        </div>
        <div class="gauge-value" style="color: {color};">{value}%</div>
        <div class="gauge-label">{gauge['name']}</div>
    </div>
    '''

html += '</div></div></div>'

display(HTML(html))

---

## UI Component Reference

The tool outputs include `_ui` hints that map to components:

| Tool | Component Hint | Renders As |
|------|----------------|------------|
| `query_service_metrics` | `ServiceDashboard` | Stat cards, area charts, gauges, sparklines |
| `get_active_alerts` | `AlertDashboard` | Alert cards with badges, sortable table, mini sparklines |
| `execute_runbook` | `RunbookExecution` | Stepper/timeline, progress bar, action buttons |
| `investigate_incident` | `InvestigationReport` | Timeline, donut chart, confidence meter, action cards |
| `send_notification` | `NotificationConfirmation` | Message preview card, delivery status |

### Data Patterns for React/Shadcn

**Charts (Recharts compatible):**
```json
{"type": "area", "data": [{"time": "14:00", "value": 250}], "color": "#ef4444"}
```

**Badges:**
```json
{"label": "Critical", "color": "#ef4444", "bg": "#fef2f2", "icon": "alert-circle"}
```

**Action Buttons:**
```json
{"label": "Execute", "action": "execute_runbook", "variant": "default", "icon": "play"}
```

---

## References

| Resource | URL |
|----------|-----|
| AG-UI Protocol | https://github.com/ag-ui-protocol/ag-ui |
| CopilotKit | https://copilotkit.ai |
| Shadcn/ui | https://ui.shadcn.com |
| Recharts | https://recharts.org |
| Lucide Icons | https://lucide.dev |